In [ ]:
import jnius
GF = jnius.autoclass("kcss.comms.math.GaloisField")
RS = jnius.autoclass("kcss.comms.error_codes.ReedSolomonCode")


In [ ]:
from pprint import pprint
import numpy as np

In [ ]:
rs = RS(3, 7, 0, 3, [1,0,1,1])
info = [1,2,3,0,0,0,0]
out = [0] * 4
rs.gf.getRem(info, rs.generator, out )
print("Remainder = " + str(out))

# introduce errors


In [ ]:
rs = RS(3, 7, 0, 3, [1,0,1,1])
codeword = [1,2,3,0,0,1,3]
syndrome = rs.calculateSyndrome(codeword)
print("Syndrome of codeword should be all zeros = " + str(syndrome))

# ------------------ Introduce error  ----------------------
c2 = codeword[:]
error_loc = np.random.randint(0,7)
c2[error_loc] = 4 + np.random.randint(0,3)
if True:
    error_loc2 = np.random.randint(0,7)
    c2[error_loc2] = 4 + np.random.randint(0,3)
    error_loc = np.unique([error_loc, error_loc2])
#c2[3] = 3
#c2[3] = 3

syndrome2 = rs.calculateSyndrome(c2)
print("Syndrome 2 = " + str(syndrome2))

# error location
err_loc1 = rs.errorLocPolyBerlekamp(syndrome2)
print("Error location1 = " + str(err_loc1))
err_loc2 = rs.errorLocPolyGauss(syndrome2)
print("Error Location 2 = " + str(err_loc2))

# -----------------  find roots  -------------------------
roots = rs.findRoots(err_loc1[-1][:])
roots = rs.findRoots(err_loc2)
print("\nRoots = " + str(roots))
print("Ground truth error location = " + str(error_loc))
est_locs = np.where(roots)[0]
print("Identified error locations = " + str(est_locs))

# -----------------  error value  ----------------------
print("Codeword = " + str(codeword))
print("Received = " + str(c2))
print("binAddTable = " + str(rs.gf.binaryAddTable))
print("binAddLUT = " + str(rs.gf.binaryAddLUT))
#vand = rs.errorValueVandermonde(syndrome2, est_locs)
forney = rs.errorValueForney(err_loc2, est_locs.tolist(), syndrome2)
print("Forney estimate = " + str(forney))

corrected = c2[:]
for ind in range(len(est_locs)):
    corrected[est_locs[ind]] = corrected[est_locs[ind]] ^ forney[ind]
print("Corrected = " + str(corrected))

# Forney
Error location polynomial 
Gamma(x) = 1 + lambda_1*x^1 + ... + lambda_v * x^v
Gamma_prime(x) = 1*lambda_1*x^(i-1) + ... + v* lambda_v*x^(v-1)
Omega(x) = S(x)* Gamma(x) (mod x^2t)
ej = Omega(Xj^-1) / Gamma_prime(Xj^-1) # for FCR=1


# Direction Solution
S_i = sum(yj * zj^i) for j = 1 2,...,numErrors
where 
* i = FCR
* yj are the error values we want to solve

Set numError set of non-linear equations
S1 = y1 * (z1^FCR)    + y2 * (z2^FCR)
S2 = y1 * (z1^(FCR+1) + y2 * (z2^(FCR+1))

## Solve error values
* Let
  * A = [[z1^(FCR), z2^FCR];
         [z1^(FCR+1), z2^(FCR+1)]
  * Y = [y1; y2]
  * S = [s1; s2] # partial syndrome

### Using Cramer's rule
Given, A*Y = S,
y1 = det(Ai)/det(A) 
where Ai = [A with the i-th column replaced by S]





In [ ]:
numErrors = len(est_locs)
n = (rs.gf.getNumElements()-1)
fcr = 1
errValues = [0]*numErrors
print("Number of errors = %d"%numErrors)
print("Syndrome = " + str(syndrome2))
# ------------------  error value calculation --------------
if numErrors == 1:
    errValues[0] = rs.gf.binaryAddTable[(syndrome2[0] - (est_locs[0] + 1) - 1) %n + 1]
#for ind in range(numErrors):
#    syndrome2[ind]

# ----------------------  apply correction  -----------------
c3 = c2[:] # copy the received word
for ind in range(numErrors):
    c3[est_locs[ind]] = c3[est_locs[ind]] ^ errValues[ind]

# -------------------  display  ----------------------------
print("Error Values = " + str(errValues))
print("Error Locations = " + str(est_locs))
print("Actual value = " + str(codeword))
print("Received Values = " + str(c2))
print("Corrected value = " + str(c3))

In [ ]:
print("Syndrome = " + str(syndrome2))
print(rs.gf.binaryAddTable)
print(rs.gf.binaryAddLUT)

est_loc = rs.getErrorLocations(roots)
print("Estimate Location = " + str(est_loc))

err_val = rs.errorValue(syndrome2, est_loc)
print("Error Value = " + str(err_val))

c3 = c2[:]
rs.correctError(c3, est_loc, err_val)
print("Actual = " + str(codeword))
print("Received = " + str(c2))
print("Corrected = " + str(c3))

# Testing GF


In [ ]:
add = rs.gf.addIndex
mult = rs.gf.multiplyIndex
div = rs.gf.divideIndex
det = rs.gf.determinant

print(rs.gf.binaryAddTable)
print(mult(4,3))
print("Determinant = "+ str(det([[1, 4],[2, 1]])))
print(div(7,3))




In [ ]:
rs = RS(9, 15, 0, 4, [1,1,0,0,1]  )
print("Generator = " + str(rs.generator))
print("binaryAddTable = " + str(rs.gf.binaryAddTable))
print("binaryAddLUT = " + str(rs.gf.binaryAddLUT))